# Bloom filters

> Keep items in memory without actually keeping them in memory.

Bloom filter is a lightweight unidirectional data structure to estimate weather or not a new item in stream has previously been seen or not. Entire needed memory is allocated during bloom filter initiation and therefore memory usage will remain contant upon encountering new items in stream.

Idea is simple - we will initiate a bit vector of length $m$ and items in data stream will be hashed into integer values and respective bit in vector will be flipped to TRUE. Since this approach will induce hash collisions, we will use multiple distinct hash functions to hash $k$ bits, where $k$ is equal to total number of distinct hash functions. We can then query our structure by hashing an element with $k$ chosen functions and only if all resulting bitvector locations are TRUE, can we estimate that item has been seen before in the stream.

Since the data structure is designed to handle collisions, then it is ideal to use fast non-cryptographic hash functions. Murmur3 is a popular choice as we can actually derive any number of distinct integer hashes from two distinct base hashes. Since murmur3 outputs 128bit hashes in two 64bit parts, then we can simply use this as base hash and generate any number of distinct hash values by using the transformation method below.

* https://www.eecs.harvard.edu/~michaelm/postscripts/rsa2008.pdf

In [18]:
import(
    "github.com/spaolacci/murmur3"
)

func string2hash(data string) [2]uint64 {
    return hasher([]byte(data))
}

func hasher(data []byte) [2]uint64 {
    hash := murmur3.New128()
    hash.Write(data)
    h1, h2 := hash.Sum128()
    return [2]uint64{
        h1, h2,
    }
}

// https://www.eecs.harvard.edu/~michaelm/postscripts/rsa2008.pdf
func transformHashes(h1, h2, i, size uint64) uint64 {
  return  ( h1 + i * h2 ) % size
}

I will also define a helper we will need later.

In [19]:
func round(f float64) float64 {
  return math.Floor(f + .5)
}

Bloom filter values $k$ and $m$ can be estimated by using the formula that is implemented below. For this purpose, we will have to define desired false positive rate $p$ and maximum number of distinct items $n$.

In [20]:
import (
	"math"
)

// BloomFilter is bitvector of length m elements
// items will be hashed to integers with k non-cryptographic functions
// boolean values in corresponding positions will be flipped
type BloomFilter struct {
	m    uint64
	k    uint64
	bits []bool
}

// InitBloomWithEstimate instantiates a new bloom filter with user defined estimate parameters
// n = number of elements in data set
// p = acceptable false positive 0 < p < 1 (no checks atm)
// hash = hashing method to use ( <= 1 for murmur, 2 for fnv, else mix of both)
func InitBloomWithEstimate(n uint, p float64) (b *BloomFilter, err error) {
	m, k := estimateBloomSize(n, p)
	b = &BloomFilter{
		m:    m,
		k:    k,
	}
	b.bits = make([]bool, m)
	return b, err
}

// m = estimated size of bloom filter array
// m = -1 * float64(n) * math.Log(p) / math.Pow(math.Log(2), 2)
// k = num of needed hash functions
func estimateBloomSize(n uint, p float64) (m, k uint64) {
	size := math.Ceil(-1 * float64(n) * math.Log(p) / math.Pow(math.Log(2.0), 2.0))
	k = uint64(round(math.Log(2.0) * size / float64(n)))
	m = uint64(size)
	return
}

// integer values
func (b *BloomFilter) genLocs(data []byte) (locations []uint64) {
	locations = make([]uint64, b.k)
	h := hasher(data)
	for i := uint64(0); i < b.k; i++ {
		locations[i] = transformHashes(h[0], h[1], i, b.m)
	}
	return
}

// Add method adds new element to bloom filter
func (b *BloomFilter) Add(data []byte) *BloomFilter {
	for _, elem := range b.genLocs(data) {
		b.bits[elem] = true
	}
	return b
}

// AddString converts item to []byte and returns Add()
func (b *BloomFilter) AddString(data string) *BloomFilter {
	return b.Add([]byte(data))
}

// Query returns the presence boolean of item from filter
// one missing bit is enough to verify non-existence
func (b *BloomFilter) Query(data []byte) bool {
	for _, elem := range b.genLocs(data) {
		if b.bits[elem] == false {
			return false
		}
	}
	return true
}

// QueryString converts item to []byte and returns Query()
func (b *BloomFilter) QueryString(data string) bool {
	return b.Query([]byte(data))
}

In [33]:
import(
    "fmt"
)

maxNoItems := 10
maxFalsePos := 0.01
bloom, err := InitBloomWithEstimate(uint(maxNoItems), maxFalsePos)

fmt.Println("Initiated bloom filter with ", bloom.k, "hash functions and ", bloom.m, "bits in vector")

Initiated bloom filter with  7 hash functions and  96 bits in vector
69
<nil>


In [34]:
bloom.AddString("8.8.8.8")
bloom.AddString("8.8.4.4")
bloom.AddString("fe80::1")

&{96 7 [false true false false false false true false false false false false true false false false false false false false false false false false false false false true false false false true false false false false false true false true false true false false false false true false false false true true false true false false true false false false false true false false false false true false true false false true false false false false false false false false false false false true false false false false false true false false false false true false]}


In [35]:
bloom.QueryString("8.8.8.8")

true


In [36]:
bloom.QueryString("192.168.1.1")

false


> PS! The most important property of bloom filters is that while a query can produce false positives, it can never produce a false negative. I.e., if any $k$ bits is FALSE, then the item has not been observed in the data stream. Thus why it's called a filter - it is often used as low overhead preprocessor for deciding weather or not to execute an expensive database query.